In [1]:
from mpl_toolkits.mplot3d import Axes3D
from rtree import index
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import math

import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [2]:
%store -r mc_data

In [3]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        xlayer_model = load_model('./models/xlayer_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        track_classifier = load_model('./models/track_classifier.h5')

In [3]:
###### Data columns
i = 0
for val in mc_data.columns:
    print(i," ",val)
    i = i + 1

0   MCParticleID
1   MC_Hit_X
2   MC_Hit_Y
3   MC_Hit_Z
4   PrHit_LHCbID
5   PrHit_Xat0
6   PrHit_Zat0
7   PrHit_isX
8   PrHit_planeCode
9   PrHit_w2
10   PrHit_yMax
11   PrHit_yMin
12   PrHit_zone
13   event
14   hasT
15   isInUT
16   isInVelo
17   isLong
18   isLong_fromB
19   isLong_fromB_more5
20   isLong_more5


In [4]:
mc_hits = mc_data.values

In [5]:
%store -r zLays
%store -r max_x
%store -r min_x
%store -r max_y
%store -r min_y
%store -r events

In [6]:
print(events)

[15837 15838 15839 15840 15841 15842 15843 15844 15845 15846 15847 15848
 15849 15850 15851 15852 15853 15854 15855 15856 15857 15858 15859 15860
 15861 15862 15863 15864 15865 15866 15867 15868 15869 15870 15871 15872
 15873 15874 15875 15876 15877 15878 15879 15880 15881 15882 15883 15884
 15885 15886 15887 15888 15889 15890 15891 15892 15893 15894 15895 15896
 15897 15898 15899 15900 15901 15902 15903 15904 15905 15906 15907 15908
 15909 15910 15911 15912 15913 15914 15915 15916 15917 15918 15919 15920
 15921 15922 15923 15924 15925 15926 15927 15928 15929 15930 15931 15932
 15933 15934 15935 15936 15937 15938 15939 15940 15941 17909 17910 17911
 17912 17913 17914 17915 17916 17917 17918 17919 17920 17921 17922 17923
 17924 17925 17926 17927 17928 17929 17930 17931 17932 17933 17934 17935
 17936 17937 17938 17939 17940 17941 17942 17943 17944 17945 17946 17947
 17948 17949 17950 17951 17952 17953 17954 17955 17956 17957 17958 17959
 17960 17961 17962 17963 17964 17965 17966 17967 17

In [7]:
event_threshold = 15840

In [8]:
def predictHeight(x0, x3, x_stereo, stereo_layer, first_layer, last_layer):
    
    z_stereo = zLays[stereo_layer]
    
    if (stereo_layer % 2 != 0):
        angle = 5
    else:
        angle = -5
        
    if (float(x3 - x0)  == 0):
        m = 1
    else:
        m = float(zLays[last_layer] - zLays[first_layer]) / float(x3 - x0)
    n = zLays[last_layer] - (m * x3)
    
    x_pred = (z_stereo - n) / m 
    beta = (x_pred - x_stereo) / (math.radians(angle) * z_stereo)
    return math.tan(beta) * z_stereo

In [54]:
#
# Test the NN model by using events not feeded as train 
#

def trackReconstruction(hits_total, index_tree):

    # Variables used for calculating metrics
    total_found = 0
    total = 0
    mean_size = np.zeros(3)

    # Variable defining the maximum number of hits gathered from the R-trees when using the predicted value from the NN
    max_hits_gathered = 12

    # Range in which valid hits are look up in the first R-tree
    long_range = 300

    X_classifier = []
    Y_classifier = []

    # For each particle of a given event
    for particle in hits_total:

        # Sometimes there are no PrHits for a given particle, so skip these ones
        if (len(particle[5]) > 1) and (particle[0] != 11) and (particle[0] != -11):

            found_num = 0            
            X_final = {}
            Zone_final = {}
            actual = -1

            # Extract all PrHits
            for i in range(0, len(particle[5])):
                if (actual != particle[8][i]):
                    X_final[particle[8][i]] = []
                    X_final[particle[8][i]].append(particle[5][i])
                    Zone_final[particle[8][i]] = particle[12][i]
                    actual = particle[8][i]
                    found_num += 1
                else:
                    X_final[actual].append(particle[5][i])

            # Check if there are minimum 12 hits found, as for the test, I am only working with complete tracks
            if not 0 in X_final or not 4 in X_final or not 8 in X_final:
                continue

            # List used to iterate over valid height found later on
            valid_y = {}

            # For each station
            for station in range(0,3):

                valid_y[station] = []

                # Compute the layer number based on station
                x0_layer = 0
                if (station == 1):
                    x0_layer = 4
                elif station == 2:
                    x0_layer = 8

                u_layer = x0_layer + 1
                v_layer = x0_layer + 2
                x1_layer = x0_layer + 3

                first_hit = X_final[x0_layer][0]
                zone = Zone_final[x0_layer]
                m = zLays[x0_layer] / first_hit
                n = zLays[x0_layer] - (m * first_hit)

                #predicted = (xlayer_model.predict(np.array([first_hit, x0_layer]).reshape(1,-1)))[0]
                #mean_predicted = ((first_hit + predicted[0] + predicted[1] + predicted[2]) / 4)

                predicted = (zLays[1] - n) / m 

                found = list(index_tree.nearest((predicted - 80, predicted + 80, u_layer, u_layer, zone, zone), objects='raw'))
                found_layer1 = sorted(found, key=lambda x: np.abs(x[1] - predicted))[:max_hits_gathered]
                arr_layer1 = np.unique(np.array(found_layer1), axis=0)

                predicted = (zLays[2] - n) / m 

                found = list(index_tree.nearest((predicted - 80, predicted + 80, v_layer, v_layer, zone, zone), objects='raw'))
                found_layer2 = sorted(found, key=lambda x: np.abs(x[1] - predicted))[:max_hits_gathered]
                arr_layer2 = np.unique(np.array(found_layer2), axis=0)

                predicted = (zLays[3] - n) / m 

                found = list(index_tree.nearest((predicted - 90, predicted + 90, x1_layer, x1_layer, zone, zone), objects='raw'))
                found_layer3 = sorted(found, key=lambda x: np.abs(x[1]- predicted))[:max_hits_gathered]
                arr_layer3 = np.unique(np.array(found_layer3), axis=0)

                max_zeros = 0
                for h3 in arr_layer3:

                    for h2 in arr_layer2:

                        # Predict height for each hit selected in the V layer
                        pred_h2 = predictHeight(first_hit, h3[1], h2[1], v_layer, x0_layer, x1_layer)

                        if (pred_h2 > 0) and (pred_h2 > max_y) or (pred_h2 < 0) and (pred_h2 < min_y):
                            continue
                            
                        if (pred_h2 > 0) and h2[2] == 0 or (pred_h2 < 0) and h2[2] == 1:
                            continue

                        # And for each hit in the V layer, we look up at hits in the U layer with the same height
                        for h1 in arr_layer1:

                            # Predict height for hit in the U layer
                            pred_h1 = predictHeight(first_hit, h3[1], h1[1], u_layer, x0_layer, x1_layer)
                            if (pred_h1 > 0) and (pred_h1 > max_y) or (pred_h1 < 0) and (pred_h1 < min_y):
                                continue
                                
                            if (pred_h1 > 0) and h1[2] == 0 or (pred_h1 < 0) and h1[2] == 1:
                                continue

                            if np.abs(pred_h2 - pred_h1) < 50:
                                # Insert into a new R-tree to enable search by height
                                # Store the valid height for later iterate over it
                                track = [first_hit, h1[1], pred_h1, h2[1], pred_h2, h3[1], station]
                                if h2[0] in particle[4] and h3[0] in particle[4]:
                                    X_classifier.append(track)
                                    Y_classifier.append(1)
                                    total_found += 1
                                elif max_zeros < 3:
                                    X_classifier.append(track)
                                    Y_classifier.append(0)
                                    max_zeros += 1
                                    
            total += 1
    #print("\nMean Total Found: %f" % (total_found/total))
    #print("\nMean Size: %f" % (mean_size[0]/total))
    return X_classifier, Y_classifier, total_found/total, mean_size[0]/total
    

In [55]:
mean = size = done = 0
test_events = events[10:300]

X_classifier = []
Y_classifier = []

p = index.Property()
p.dimension = 3
    
for event in test_events:
    particles_test_event = mc_hits[mc_hits[:,13] == event]
    # print("Event length %d " % len(particles_test_event))
    idx = {}
    mean_val = size_val = length_valid = 0
    hit_tree = index.Index(properties=p, interleaved=False)
    for j in range(0, len(particles_test_event)):
        hit = particles_test_event[j]
        if (len(hit[5]) > 1) and (hit[0] != 11) and (hit[0] != -11):
            length_valid += 1
            for i in range(0,len(hit[5])):
                if (i > 0  and np.abs(hit[5][i] - hit[5][i - 1]) < 2):
                    continue
                hit_store = [hit[4][i], hit[5][i], hit[12][i], j]
                hit_tree.insert(int(hit[4][i]), (float(hit[5][i]), float(hit[5][i]), hit[8][i], hit[8][i], hit[12][i], hit[12][i]), hit_store)
    
    X, Y, mean_val, size_val = trackReconstruction(particles_test_event, hit_tree)
    
    X_classifier += X
    Y_classifier += Y
    mean += mean_val
    size += size_val
    done += 1
    print("Event %d/%d \t Acc: %f Size %f Proportion %f" % (done,len(test_events), mean_val, size_val, Y.count(0)/(Y.count(1) + 1)))
    
print("\nFinal Result:")
print("\t X_Size : %f" % (len(X_classifier)))
print("\t Y_Size: %f" % (len(Y_classifier)))
print("\t Y count 0 : %f" % (Y_classifier.count(0)))
print("\t Y count 1: %f" % (Y_classifier.count(1)))
print("\t Mean: %f" % (mean/len(test_events)))
print("\t Size: %f" % (size/len(test_events)))

Event 1/290 	 Acc: 1.333333 Size 0.000000 Proportion 0.303030
Event 2/290 	 Acc: 1.370968 Size 0.000000 Proportion 0.860465
Event 3/290 	 Acc: 1.063830 Size 0.000000 Proportion 0.705882
Event 4/290 	 Acc: 2.085714 Size 0.000000 Proportion 0.351351
Event 5/290 	 Acc: 2.181818 Size 0.000000 Proportion 0.896552
Event 6/290 	 Acc: 1.966667 Size 0.000000 Proportion 0.806723
Event 7/290 	 Acc: 2.977778 Size 0.000000 Proportion 0.713755
Event 8/290 	 Acc: 4.058824 Size 0.000000 Proportion 1.293421
Event 9/290 	 Acc: 1.325581 Size 0.000000 Proportion 0.482759
Event 10/290 	 Acc: 2.791367 Size 0.000000 Proportion 1.367609
Event 11/290 	 Acc: 1.500000 Size 0.000000 Proportion 0.983607
Event 12/290 	 Acc: 3.650980 Size 0.000000 Proportion 1.469957
Event 13/290 	 Acc: 1.510870 Size 0.000000 Proportion 1.214286
Event 14/290 	 Acc: 2.082353 Size 0.000000 Proportion 1.039326
Event 15/290 	 Acc: 2.263158 Size 0.000000 Proportion 1.046154
Event 16/290 	 Acc: 4.100840 Size 0.000000 Proportion 1.544710
E

Event 131/290 	 Acc: 3.369048 Size 0.000000 Proportion 1.410935
Event 132/290 	 Acc: 2.202128 Size 0.000000 Proportion 1.620192
Event 133/290 	 Acc: 3.852632 Size 0.000000 Proportion 1.362892
Event 134/290 	 Acc: 2.705882 Size 0.000000 Proportion 1.158845
Event 135/290 	 Acc: 2.132353 Size 0.000000 Proportion 1.109589
Event 136/290 	 Acc: 2.192771 Size 0.000000 Proportion 1.448087
Event 137/290 	 Acc: 2.341463 Size 0.000000 Proportion 1.264249
Event 138/290 	 Acc: 2.850746 Size 0.000000 Proportion 1.347258
Event 139/290 	 Acc: 2.259740 Size 0.000000 Proportion 0.874286
Event 140/290 	 Acc: 3.540230 Size 0.000000 Proportion 1.684324
Event 141/290 	 Acc: 2.517647 Size 0.000000 Proportion 1.325581
Event 142/290 	 Acc: 2.865079 Size 0.000000 Proportion 1.337017
Event 143/290 	 Acc: 3.712644 Size 0.000000 Proportion 1.417311
Event 144/290 	 Acc: 2.147727 Size 0.000000 Proportion 1.478947
Event 145/290 	 Acc: 2.861111 Size 0.000000 Proportion 1.808140
Event 146/290 	 Acc: 3.987730 Size 0.000

Event 260/290 	 Acc: 2.126050 Size 0.000000 Proportion 1.559055
Event 261/290 	 Acc: 4.367876 Size 0.000000 Proportion 1.223934
Event 262/290 	 Acc: 2.071429 Size 0.000000 Proportion 1.951890
Event 263/290 	 Acc: 3.381579 Size 0.000000 Proportion 1.582524
Event 264/290 	 Acc: 1.758065 Size 0.000000 Proportion 1.009091
Event 265/290 	 Acc: 3.393103 Size 0.000000 Proportion 1.296146
Event 266/290 	 Acc: 1.805556 Size 0.000000 Proportion 0.931298
Event 267/290 	 Acc: 2.410526 Size 0.000000 Proportion 1.286957
Event 268/290 	 Acc: 1.500000 Size 0.000000 Proportion 0.606557
Event 269/290 	 Acc: 1.117647 Size 0.000000 Proportion 0.200000
Event 270/290 	 Acc: 2.554688 Size 0.000000 Proportion 1.673780
Event 271/290 	 Acc: 1.885714 Size 0.000000 Proportion 0.969925
Event 272/290 	 Acc: 1.977273 Size 0.000000 Proportion 0.784091
Event 273/290 	 Acc: 3.295455 Size 0.000000 Proportion 1.403670
Event 274/290 	 Acc: 3.215517 Size 0.000000 Proportion 1.152406
Event 275/290 	 Acc: 2.500000 Size 0.000

In [56]:
print(Y_classifier.count(0)/len(X_classifier))

0.565604529361556


In [79]:
track_classifier = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
track_classifier.add(Dense(12, activation='relu', input_dim=7))
track_classifier.add(Dense(8, activation='relu'))
track_classifier.add(Dense(1, activation='sigmoid'))
track_classifier.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
track_classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 12)                96        
_________________________________________________________________
dense_20 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 9         
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


In [80]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X_classifier), np.array(Y_classifier))
print("X Train length: %d" % len(X_train))
print("X Test Length %d" % len(X_test))
print("Y Train length: %d" % len(y_train))
print("Y Test length: %d" % len(y_test))

X Train length: 148497
X Test Length 49500
Y Train length: 148497
Y Test length: 49500


In [82]:
track_classifier.fit(X_train, y_train, epochs=50, batch_size=100, verbose=1, validation_split=0.1)

Train on 133647 samples, validate on 14850 samples
Epoch 1/50
133647/133647 [==============================] - 3s 22us/sample - loss: 0.5972 - acc: 0.6791 - val_loss: 0.5934 - val_acc: 0.6841
Epoch 2/50
133647/133647 [==============================] - 3s 25us/sample - loss: 0.5883 - acc: 0.6957 - val_loss: 0.5703 - val_acc: 0.7053
Epoch 3/50
133647/133647 [==============================] - 3s 24us/sample - loss: 0.5817 - acc: 0.7038 - val_loss: 0.5603 - val_acc: 0.7111
Epoch 4/50
133647/133647 [==============================] - 3s 19us/sample - loss: 0.5712 - acc: 0.7132 - val_loss: 0.5773 - val_acc: 0.7215
Epoch 5/50
133647/133647 [==============================] - 3s 19us/sample - loss: 0.5532 - acc: 0.7312 - val_loss: 0.5720 - val_acc: 0.7220
Epoch 6/50
133647/133647 [==============================] - 3s 23us/sample - loss: 0.5374 - acc: 0.7449 - val_loss: 0.5321 - val_acc: 0.7434
Epoch 7/50
133647/133647 [==============================] - 3s 21us/sample - loss: 0.5218 - acc: 0.7571

In [63]:
score, acc = track_classifier.evaluate(X_test, y_test, batch_size=25)
print('Test score:', score)
print('Test accuracy:', acc)

49500/49500 [==============================] - 2s 42us/sample - loss: 0.3909 - acc: 0.8301
Test score: 0.39087873374422394
Test accuracy: 0.8301414


In [64]:
track_classifier.save("./models/seed_station_validation.h5")